#Install requirements

In [4]:
!pip install ipdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!git clone https://github.com/209sontung/IVGD

Cloning into 'IVGD'...
remote: Enumerating objects: 753, done.
remote: Counting objects: 100% (753/753), done.
remote: Compressing objects: 100% (661/661), done.
remote: Total 753 (delta 88), reused 733 (delta 75), pack-reused 0
Receiving objects: 100% (753/753), 11.81 MiB | 9.46 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [6]:
%cd '/content/IVGD'

/content/IVGD


#Modeling

##Graph Diffusion

In [7]:
!python pretrain.py

/content/IVGD/main/models/MLP.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if drop_prob is 0:
<Undirected, unweighted and connected SparseGraph with 474 edges (no self-loops). Data: adj_matrix (100x100), node_names (100), prob_matrix (100x100), influ_mat_list (60x100x25)>
(48, 100, 25)
(60, 100, 25)
2022-12-08 15:50:04:PyTorch seed: 765246594
2022-12-08 15:50:16:Epoch 0: Train loss = 0.1217, Train error = 0.1217, early stopping loss = 0.1217, early stopping error = 0.1217, (7.066 sec)
2022-12-08 15:50:17:Epoch 1: Train loss = 0.1202, Train error = 0.1202, early stopping loss = 0.1211, early stopping error = 0.1211, (0.797 sec)
2022-12-08 15:50:18:Epoch 2: Train loss = 0.1204, Train error = 0.1204, early stopping loss = 0.1185, early stopping error = 0.1185, (0.681 sec)
2022-12-08 15:50:18:Epoch 3: Train loss = 0.1183, Train error = 0.1183, early stopping loss = 0.1198, early stopping error = 0.1198, (0.694 sec)
2022-12-08 15:50:19:Epoch 4: Train loss = 0.1159, Train 

##Source Localization

In [8]:
import logging
import numpy as np
import copy
import torch.nn as nn
import matplotlib.pyplot as plt

from main.training import FeatureCons, get_idx_new_seeds,get_predictions_new_seeds
from main.utils import load_dataset
from main.alm_net import alm_net
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,roc_auc_score,mean_squared_error

from data.sparsegraph import SparseGraph

logging.basicConfig(
    format='%(asctime)s:%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO)
plt.style.use('seaborn')
import torch.optim as optim

# key parameters
dataset = 'covid_sample'  # 'karate','dolphins','jazz','netscience','cora_ml', 'power_grid'
model_name = 'deepis'  # 'deepis', ''
graph = load_dataset(dataset)
print(graph)
influ_mat_list = copy.copy(graph.influ_mat_list)
num_training= int(len(graph.influ_mat_list)*0.8)
graph.influ_mat_list = graph.influ_mat_list[:num_training]
print(graph.influ_mat_list.shape), print(influ_mat_list.shape)
# training parameters
ndim =5 #for simulated datasets
fea_constructor = FeatureCons(model_name, ndim=ndim)
fea_constructor.prob_matrix = graph.prob_matrix
device = 'cuda'  # 'cpu', 'cuda'
model = torch.load("i-deepis_"+dataset+".pt")
influ_pred=get_predictions_new_seeds(model,fea_constructor,graph.influ_mat_list[0,:,0],np.arange(len(graph.influ_mat_list[0,:,0])))
criterion = nn.CrossEntropyLoss()
alpha = 1
tau = 10
rho = 1e-3
lamda = 0
threshold=0.5
nu = torch.zeros(size=(graph.influ_mat_list.shape[1], 1))
net = alm_net(alpha=alpha, tau=tau, rho=rho)
optimizer = optim.SGD(net.parameters(), lr=1e-2)
net.train()
for i, influ_mat in enumerate(graph.influ_mat_list):
    print("i={:d}".format(i))
    seed_vec = influ_mat[:, 0]
    seed_idx = np.argwhere(seed_vec == 1)  # used by PIteration
    influ_vec = influ_mat[:, -1]
    fea_constructor.prob_matrix = graph.prob_matrix
    seed_preds = get_idx_new_seeds(model, influ_vec)
    seed_preds = torch.tensor(seed_preds).unsqueeze(-1).float()
    influ_vec = torch.tensor(influ_vec).unsqueeze(-1).float()
    seed_vec = torch.tensor(seed_vec).unsqueeze(-1).float()
    for epoch in range(10):
        print("epoch:" + str(epoch))
        optimizer.zero_grad()
        seed_correction = net(seed_preds, seed_vec, lamda)
        loss = criterion(seed_correction, seed_vec.squeeze(-1).long())
        print("loss:{:0.6f}".format(loss))
        loss.backward(retain_graph=True)
        optimizer.step()
net.eval()

<Undirected, unweighted and connected SparseGraph with 474 edges (no self-loops). Data: adj_matrix (100x100), node_names (100), prob_matrix (100x100), influ_mat_list (60x100x25)>
(48, 100, 25)
(60, 100, 25)
i=0
epoch:0
loss:0.355245
epoch:1
loss:0.315908
epoch:2
loss:0.315732
epoch:3
loss:0.315562
epoch:4
loss:0.315397
epoch:5
loss:0.315240
epoch:6
loss:0.315086
epoch:7
loss:0.314934
epoch:8
loss:0.314787
epoch:9
loss:0.314648
i=1
epoch:0
loss:0.320274
epoch:1
loss:0.317415
epoch:2
loss:0.316955
epoch:3
loss:0.316680
epoch:4
loss:0.316498
epoch:5
loss:0.316363
epoch:6
loss:0.316229
epoch:7
loss:0.316097
epoch:8
loss:0.315965
epoch:9
loss:0.315834
i=2
epoch:0
loss:0.314890
epoch:1
loss:0.314770
epoch:2
loss:0.314650
epoch:3
loss:0.314552
epoch:4
loss:0.314483
epoch:5
loss:0.314416
epoch:6
loss:0.314365
epoch:7
loss:0.314331
epoch:8
loss:0.314296
epoch:9
loss:0.314262
i=3
epoch:0
loss:0.319686
epoch:1
loss:0.317187
epoch:2
loss:0.316037
epoch:3
loss:0.315596
epoch:4
loss:0.315311
epoch:5

alm_net(
  (net1): correction(
    (fc1): Linear(in_features=2, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=1000, bias=True)
    (fc3): Linear(in_features=1000, out_features=2, bias=True)
  )
  (net2): correction(
    (fc1): Linear(in_features=2, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=1000, bias=True)
    (fc3): Linear(in_features=1000, out_features=2, bias=True)
  )
  (net3): correction(
    (fc1): Linear(in_features=2, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=1000, bias=True)
    (fc3): Linear(in_features=1000, out_features=2, bias=True)
  )
  (net4): correction(
    (fc1): Linear(in_features=2, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=1000, bias=True)
    (fc3): Linear(in_features=1000, out_features=2, bias=True)
  )
  (net5): correction(
    (fc1): Linear(in_features=2, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000

In [9]:
graph = load_dataset(dataset)
influ_mat_list = copy.copy(graph.influ_mat_list)
print(graph)
train_acc = 0
test_acc = 0
train_pr = 0
test_pr = 0
train_re = 0
test_re = 0
train_fs = 0
test_fs = 0
train_auc = 0
test_auc= 0

pred_sourcenode = []
sourcenode = []
influ_node = []


for i, influ_mat in enumerate(influ_mat_list):
    print("i={:d}".format(i))
    seed_vec = influ_mat[:, 0]
    seed_idx = np.argwhere(seed_vec == 1)  # used by PIteration
    influ_vec = influ_mat[:, -1]
    positive = np.where(seed_vec == 1)
    fea_constructor.prob_matrix = graph.prob_matrix
    seed_preds = get_idx_new_seeds(model, influ_vec)
    seed_preds = torch.tensor(seed_preds).unsqueeze(-1).float()
    influ_vec = torch.tensor(influ_vec).unsqueeze(-1).float()
    seed_vec = torch.tensor(seed_vec).unsqueeze(-1).float()
    seed_correction = net(seed_preds, seed_preds,lamda)
    seed_correction =torch.softmax(seed_correction,dim=1)
    seed_preds = seed_preds.squeeze(-1).detach().numpy()
    seed_correction = seed_correction[:,1].squeeze(-1).detach().numpy()
    seed_vec = seed_vec.squeeze(-1).detach().numpy()
    if i > num_training:
      pred_sourcenode.append(np.argwhere(seed_correction > 0.5))
      sourcenode.append(np.argwhere(seed_vec > 0.5))
      influ_node.append(influ_vec)
    if i < num_training:
        train_acc += accuracy_score(seed_vec, seed_correction>=threshold)
        train_pr += precision_score(seed_vec, seed_correction>=threshold,zero_division=1)
        train_re += recall_score(seed_vec, seed_correction>=threshold)
        train_fs += f1_score(seed_vec, seed_correction>=threshold)
        train_auc += roc_auc_score(seed_vec, seed_correction)
    else:
        #print(accuracy_score(seed_vec, seed_correction>=threshold))
        test_acc += accuracy_score(seed_vec, seed_correction>=threshold)
        test_pr += precision_score(seed_vec, seed_correction>=threshold,zero_division=1)
        test_re += recall_score(seed_vec, seed_correction>=threshold)
        test_fs += f1_score(seed_vec, seed_correction>=threshold)
        test_auc += roc_auc_score(seed_vec, seed_preds)

print('training acc:', train_acc / num_training)
print('training pr:', train_pr / num_training)
print('training re:', train_re / num_training)
print('training fs:', train_fs / num_training)
# print('training auc:', train_auc / num_training)
print('test acc:', test_acc / (len(influ_mat_list) - num_training))
print('test pr:', test_pr / (len(influ_mat_list) - num_training))
print('test re:', test_re / (len(influ_mat_list) - num_training))
print('test fs:', test_fs / (len(influ_mat_list) - num_training))
# print('test auc:', test_auc / (len(influ_mat_list) - num_training))

<Undirected, unweighted and connected SparseGraph with 474 edges (no self-loops). Data: adj_matrix (100x100), node_names (100), prob_matrix (100x100), influ_mat_list (60x100x25)>
i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7
i=8
i=9
i=10
i=11
i=12
i=13
i=14
i=15
i=16
i=17
i=18
i=19
i=20
i=21
i=22
i=23
i=24
i=25
i=26
i=27
i=28
i=29
i=30
i=31
i=32
i=33
i=34
i=35
i=36
i=37
i=38
i=39
i=40
i=41
i=42
i=43
i=44
i=45
i=46
i=47
i=48
i=49
i=50
i=51
i=52
i=53
i=54
i=55
i=56
i=57
i=58
i=59
training acc: 1.0
training pr: 1.0
training re: 1.0
training fs: 1.0
test acc: 1.0
test pr: 1.0
test re: 1.0
test fs: 1.0


In [10]:
def reformat(sourcenode, pred_sourcenode, influ_node):
  for i in range(len(sourcenode)):
    if len(sourcenode[i]) and len(pred_sourcenode) >1:
      sourcenode[i] = sourcenode[i].squeeze(-1)
      pred_sourcenode[i] = pred_sourcenode[i].squeeze(-1)
      influ_node[i] = influ_node[i].squeeze(-1)
  return sourcenode, pred_sourcenode, influ_node

In [11]:
sourcenode_covid = sourcenode
pred_sourcenode_covid = pred_sourcenode
influ_covid = influ_node

source_covid, pred_covid, diff_covid = reformat(sourcenode_covid, pred_sourcenode_covid, influ_covid)

#Visualization

##Covid_sample graph-based visualization

In [33]:
import pandas as pd
df = pd.read_csv('/content/IVGD/data/sample_covid.csv')
df.drop(['weight'], axis=1, inplace=True)
df

,person,contact
0,p1,p2
1,p1,p3
2,p1,p4
3,p1,p5
4,p1,p6
...,...,...
265,p98,p66
266,p99,p25
267,p99,p57
268,p99,p50


In [34]:
import networkx as nx

G = nx.from_pandas_edgelist(df,'person','contact')
G

In [35]:
ZKC_graph = G
spring_3D = nx.spring_layout(ZKC_graph, dim=3, seed=18)

In [36]:
x_nodes = [spring_3D[i][0] for i in ZKC_graph.nodes]# x-coordinates of nodes
y_nodes = [spring_3D[i][1] for i in ZKC_graph.nodes]# y-coordinates
z_nodes = [spring_3D[i][2] for i in ZKC_graph.nodes]# z-coordinates

In [37]:
#We also need a list of edges to include in the plot
edge_list = ZKC_graph.edges()

#we  need to create lists that contain the starting and ending coordinates of each edge.
x_edges=[]
y_edges=[]
z_edges=[]

#need to fill these with all of the coordiates
for edge in edge_list:
    #format: [beginning,ending,None]
    x_coords = [spring_3D[edge[0]][0],spring_3D[edge[1]][0],None]
    x_edges += x_coords

    y_coords = [spring_3D[edge[0]][1],spring_3D[edge[1]][1],None]
    y_edges += y_coords

    z_coords = [spring_3D[edge[0]][2],spring_3D[edge[1]][2],None]
    z_edges += z_coords

In [38]:
import plotly.graph_objects as go

#create a trace for the edges
trace_edges = go.Scatter3d(x=x_edges,
                        y=y_edges,
                        z=z_edges,
                        mode='lines',
                        line=dict(color='black', width=2),
                        hoverinfo='none')

In [39]:
#create a trace for the nodes
trace_nodes = go.Scatter3d(x=x_nodes,
                         y=y_nodes,
                        z=z_nodes,
                        mode='markers',
                        marker=dict(symbol='circle',
                                    size=10,
                                    color='darkgreen', #color the nodes according to their community
                                    colorscale=['lightgreen','magenta'], #either green or mageneta
                                    line=dict(color='black', width=0.5)),
                        # text=club_labels,
                        hoverinfo='text', opacity=0.4)

In [40]:
#we need to set the axis for the plot 
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')

In [41]:
#also need to create the layout for our plot
layout = go.Layout(title="Covid_sample",
                width=650,
                height=625,
                showlegend=False,
                scene=dict(xaxis=dict(axis),
                        yaxis=dict(axis),
                        zaxis=dict(axis),
                        ),
                margin=dict(t=100),
                hovermode='closest')

In [42]:
#Include the traces we want to plot and create a figure
data = [trace_edges, trace_nodes]
fig = go.Figure(data=data, layout=layout)

fig.show()

##Graph Diffusion 

In [43]:
def draw_diffusion(G, diff_node):
  person = ['person'+str(i) + ': ' for i in range(1,100)]
  k_ = (1-np.array(diff_node))*10
  k = np.array(diff_node).astype('str').tolist()
  diff_values = list(map(str.__add__, person, k))
  ZKC_graph = G
  spring_3D = nx.spring_layout(ZKC_graph, dim=3, seed=18)
  x_nodes = [spring_3D[i][0] for i in ZKC_graph.nodes]# x-coordinates of nodes
  y_nodes = [spring_3D[i][1] for i in ZKC_graph.nodes]# y-coordinates
  z_nodes = [spring_3D[i][2] for i in ZKC_graph.nodes]# z-coordinates
  #We also need a list of edges to include in the plot
  edge_list = ZKC_graph.edges()

  #we  need to create lists that contain the starting and ending coordinates of each edge.
  x_edges=[]
  y_edges=[]
  z_edges=[]

  #need to fill these with all of the coordiates
  for edge in edge_list:
      #format: [beginning,ending,None]
      x_coords = [spring_3D[edge[0]][0],spring_3D[edge[1]][0],None]
      x_edges += x_coords

      y_coords = [spring_3D[edge[0]][1],spring_3D[edge[1]][1],None]
      y_edges += y_coords

      z_coords = [spring_3D[edge[0]][2],spring_3D[edge[1]][2],None]
      z_edges += z_coords

  trace_edges = go.Scatter3d(x=x_edges,
                        y=y_edges,
                        z=z_edges,
                        mode='lines',
                        line=dict(color='black', width=2),
                        hoverinfo='none')
  
    #create a trace for the nodes
  trace_nodes = go.Scatter3d(x=x_nodes,
                          y=y_nodes,
                          z=z_nodes,
                          mode='markers',
                          marker=dict(symbol='circle',
                                      size=10,
                                      color=k_, #color the nodes according to their community
                                      colorscale='PiYG', #either green or mageneta
                                      line=dict(color='black', width=0.5)),
                          text=diff_values,
                          hoverinfo='text', opacity=0.7)
  
    #we need to set the axis for the plot 
  axis = dict(showbackground=False,
              showline=False,
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='')
  #also need to create the layout for our plot
  layout = go.Layout(title="The diffusion of covid_sample",
                  width=650,
                  height=625,
                  showlegend=False,
                  scene=dict(xaxis=dict(axis),
                          yaxis=dict(axis),
                          zaxis=dict(axis),
                          ),
                  margin=dict(t=100),
                  hovermode='closest')
  #Include the traces we want to plot and create a figure
  data = [trace_edges, trace_nodes]
  fig = go.Figure(data=data, layout=layout)

  fig.show()

In [44]:
draw_diffusion(G, diff_covid[7])

##Sources Localization

In [47]:
def draw_source(G, diff_node):
  person = ['person'+str(i) + ': ' for i in range(1,100)]
  diff_node = diff_node.tolist()
  for i in range(len(diff_node)):
    if diff_node[i] < 1:
      diff_node[i] = 0
  k_ = (1-np.array(diff_node))*10
  k = np.array(diff_node).astype('str').tolist()
  diff_values = list(map(str.__add__, person, k))
  ZKC_graph = G
  spring_3D = nx.spring_layout(ZKC_graph, dim=3, seed=18)
  x_nodes = [spring_3D[i][0] for i in ZKC_graph.nodes]# x-coordinates of nodes
  y_nodes = [spring_3D[i][1] for i in ZKC_graph.nodes]# y-coordinates
  z_nodes = [spring_3D[i][2] for i in ZKC_graph.nodes]# z-coordinates
  #We also need a list of edges to include in the plot
  edge_list = ZKC_graph.edges()

  #we  need to create lists that contain the starting and ending coordinates of each edge.
  x_edges=[]
  y_edges=[]
  z_edges=[]

  #need to fill these with all of the coordiates
  for edge in edge_list:
      #format: [beginning,ending,None]
      x_coords = [spring_3D[edge[0]][0],spring_3D[edge[1]][0],None]
      x_edges += x_coords

      y_coords = [spring_3D[edge[0]][1],spring_3D[edge[1]][1],None]
      y_edges += y_coords

      z_coords = [spring_3D[edge[0]][2],spring_3D[edge[1]][2],None]
      z_edges += z_coords

  trace_edges = go.Scatter3d(x=x_edges,
                        y=y_edges,
                        z=z_edges,
                        mode='lines',
                        line=dict(color='black', width=2),
                        hoverinfo='none')
  
    #create a trace for the nodes
  trace_nodes = go.Scatter3d(x=x_nodes,
                          y=y_nodes,
                          z=z_nodes,
                          mode='markers',
                          marker=dict(symbol='circle',
                                      size=10,
                                      color=k_, #color the nodes according to their community
                                      colorscale='PiYG', #either green or mageneta
                                      line=dict(color='black', width=0.5)),
                          text=diff_values,
                          hoverinfo='text', opacity=0.65)
  
    #we need to set the axis for the plot 
  axis = dict(showbackground=False,
              showline=False,
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='')
  #also need to create the layout for our plot
  layout = go.Layout(title="The source localization of covid_sample",
                  width=650,
                  height=625,
                  showlegend=False,
                  scene=dict(xaxis=dict(axis),
                          yaxis=dict(axis),
                          zaxis=dict(axis),
                          ),
                  margin=dict(t=100),
                  hovermode='closest')
  #Include the traces we want to plot and create a figure
  data = [trace_edges, trace_nodes]
  fig = go.Figure(data=data, layout=layout)

  fig.show()

In [48]:
draw_source(G, diff_covid[7])